# 📥 Data Pull: OpenAQ API

This notebook pulls air quality measurements from the OpenAQ API and saves them locally for further analysis in downstream notebooks.


In [ ]:
# Imports and Environment Setup
 
# Standard libraries
import os
import sys
import json
import requests
import pandas as pd

# Add parent directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import API keys securely from config file, which loads them from the .env file
from scripts.config import OPENAQ_API_KEY, GITHUB_PAT

# define headers for API call
headers = {
    "accept": "application/json",
    "X-API-Key": OPENAQ_API_KEY
}

In [ ]:
# make a simple API Call
url = "https://api.openaq.org/v3/countries"

response = requests.get(url, headers=headers)
response.raise_for_status()  # raises error if something went wrong

data = response.json()
print(data['results'][:3])  # print first 3 results

[{'id': 1, 'code': 'ID', 'name': 'Indonesia', 'datetimeFirst': '2016-01-30T01:00:00Z', 'datetimeLast': '2025-07-29T18:05:21.737000Z', 'parameters': [{'id': 1, 'name': 'pm10', 'units': 'µg/m³', 'displayName': None}, {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': None}, {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'displayName': None}, {'id': 10, 'name': 'o3', 'units': 'ppm', 'displayName': None}, {'id': 11, 'name': 'bc', 'units': 'µg/m³', 'displayName': None}, {'id': 15, 'name': 'no2', 'units': 'ppb', 'displayName': None}, {'id': 19, 'name': 'pm1', 'units': 'µg/m³', 'displayName': None}, {'id': 21, 'name': 'co2', 'units': 'ppm', 'displayName': None}, {'id': 24, 'name': 'no', 'units': 'ppb', 'displayName': None}, {'id': 98, 'name': 'relativehumidity', 'units': '%', 'displayName': None}, {'id': 100, 'name': 'temperature', 'units': 'c', 'displayName': None}, {'id': 125, 'name': 'um003', 'units': 'particles/cm³', 'displayName': None}, {'id': 128, 'name': 'temperature', 'units': 'f'

In [ ]:
'''Start by listing locations or sensors in US with PM2.5 parameter
This will give valid location or sensor IDs to target:
'''

url_locations = "https://api.openaq.org/v3/locations"
params_locations = {
    "country": "US",
    "parameters": "pm25",
    "limit": 5
}

response = requests.get(url_locations, headers=headers, params=params_locations)
response.raise_for_status()
locations_data = response.json()
print(locations_data['results'])

[{'id': 3, 'name': 'NMA - Nima', 'locality': None, 'timezone': 'Africa/Accra', 'country': {'id': 152, 'code': 'GH', 'name': 'Ghana'}, 'owner': {'id': 4, 'name': 'Unknown Governmental Organization'}, 'provider': {'id': 209, 'name': 'Dr. Raphael E. Arku and Colleagues'}, 'isMobile': False, 'isMonitor': True, 'instruments': [{'id': 2, 'name': 'Government Monitor'}], 'sensors': [{'id': 6, 'name': 'pm10 µg/m³', 'parameter': {'id': 1, 'name': 'pm10', 'units': 'µg/m³', 'displayName': 'PM10'}}, {'id': 5, 'name': 'pm25 µg/m³', 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5'}}], 'coordinates': {'latitude': 5.58389, 'longitude': -0.19968}, 'licenses': None, 'bounds': [-0.19968, 5.58389, -0.19968, 5.58389], 'distance': None, 'datetimeFirst': None, 'datetimeLast': None}, {'id': 4, 'name': 'NMT - Nima', 'locality': None, 'timezone': 'Africa/Accra', 'country': {'id': 152, 'code': 'GH', 'name': 'Ghana'}, 'owner': {'id': 4, 'name': 'Unknown Governmental Organization'}, '

In [94]:
location_ids = [loc['id'] for loc in locations_data['results']]
print("Location IDs:", location_ids)

Location IDs: [3, 4, 5, 6, 7]


In [102]:
# call the parameters endpoint to list all available parameters

url_parameters = "https://api.openaq.org/v3/parameters"
response = requests.get(url_parameters, headers=headers)
response.raise_for_status()
parameters_data = response.json()
print(parameters_data['results'])


[{'id': 1, 'name': 'pm10', 'units': 'µg/m³', 'displayName': 'PM10', 'description': 'Particulate matter less than 10 micrometers in diameter mass concentration'}, {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5', 'description': 'Particulate matter less than 2.5 micrometers in diameter mass concentration'}, {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'displayName': 'O₃ mass', 'description': 'Ozone mass concentration'}, {'id': 4, 'name': 'co', 'units': 'µg/m³', 'displayName': 'CO mass', 'description': 'Carbon Monoxide mass concentration'}, {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'displayName': 'NO₂ mass', 'description': 'Nitrogen Dioxide mass concentration'}, {'id': 6, 'name': 'so2', 'units': 'µg/m³', 'displayName': 'SO₂ mass', 'description': 'Sulfur Dioxide mass concentration'}, {'id': 7, 'name': 'no2', 'units': 'ppm', 'displayName': 'NO₂', 'description': 'Nitrogen Dioxide concentration'}, {'id': 8, 'name': 'co', 'units': 'ppm', 'displayName': 'CO', 'description': 'Car

In [106]:
# reshape parameters data
params_list = parameters_data['results']

# Convert to DataFrame
df_params = pd.DataFrame(params_list)

# Select and reorder columns for better readability
columns_to_show = ['id', 'name', 'displayName', 'units', 'description']
df_params = df_params[columns_to_show]

# Display the DataFrame nicely
pd.set_option('display.max_colwidth', 100)  # to avoid truncation of descriptions
print(df_params)

       id              name           displayName          units  \
0       1              pm10                  PM10          µg/m³   
1       2              pm25                 PM2.5          µg/m³   
2       3                o3               O₃ mass          µg/m³   
3       4                co               CO mass          µg/m³   
4       5               no2              NO₂ mass          µg/m³   
5       6               so2              SO₂ mass          µg/m³   
6       7               no2                   NO₂            ppm   
7       8                co                    CO            ppm   
8       9               so2                   SO₂            ppm   
9      10                o3                    O₃            ppm   
10     11                bc                    BC          µg/m³   
11     15               no2                   NO₂            ppb   
12     19               pm1                   PM1          µg/m³   
13     21               co2                   CO